In [131]:
!pip install torchvision
!pip install pandas
!pip install -U scikit-learn

In [177]:
# Cell 1: Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import csv
from sklearn.metrics import fbeta_score
from sklearn.metrics import confusion_matrix

In [188]:
# Cell 2: Define custom dataset
class ImageLabelDataset(nn.Module):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

        # Encode labels
        self.three_d_encoder = LabelEncoder()
        self.symmetric_encoder = LabelEncoder()
        self.food_encoder = LabelEncoder()
        self.person_encoder = LabelEncoder()
        self.nature_encoder = LabelEncoder()
        self.architectural_encoder = LabelEncoder()
        self.animal_encoder = LabelEncoder()

        self.data['three_d_encoded'] = self.three_d_encoder.fit_transform(self.data['3D'])
        self.data['symmetric_encoded'] = self.symmetric_encoder.fit_transform(self.data['symmetric'])
        self.data['food_encoded'] = self.food_encoder.fit_transform(self.data['food'])
        self.data['person_encoded'] = self.person_encoder.fit_transform(self.data['person'])
        self.data['nature_encoded'] = self.nature_encoder.fit_transform(self.data['nature'])
        self.data['architectural_encoded'] = self.architectural_encoder.fit_transform(self.data['architectural'])
        self.data['animal_encoded'] = self.animal_encoder.fit_transform(self.data['animal'])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.data.iloc[idx]['image_path'])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        three_d = self.data.iloc[idx]['three_d_encoded']
        symmetric = self.data.iloc[idx]['symmetric_encoded']
        food = self.data.iloc[idx]['food_encoded']
        person = self.data.iloc[idx]['person_encoded']
        nature = self.data.iloc[idx]['nature_encoded']
        architectural = self.data.iloc[idx]['architectural_encoded']
        animal = self.data.iloc[idx]['animal_encoded']

        return image, three_d, symmetric, food, person ,nature ,architectural ,animal

#transform
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# dataset
dataset = ImageLabelDataset(csv_file='Day1/dataset1/NextGenClassification/train_classification.csv', img_dir='Day1/dataset1/NextGenClassification', transform=transform)

# train and validate
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

print("จำนวนข้อมูลทั้งหมด:", len(dataset))
print("จำนวนข้อมูลสำหรับ training:", len(train_data))
print("จำนวนข้อมูลสำหรับ validation:", len(val_data))

sample_image,sample_three_d, sample_symmetric, sample_food, sample_person ,sample_nature ,sample_architectural ,sample_animal = dataset[0]
print("ขนาดของรูปภาพ:", sample_image.shape)
print("ตัวอย่าง sample_three_d (encoded):", sample_three_d)
print("ตัวอย่าง sample_symmetric (encoded):", sample_symmetric)
print("ตัวอย่าง sample_food (encoded):", sample_food)
print("ตัวอย่าง sample_person (encoded):", sample_person)
print("ตัวอย่าง sample_nature (encoded):", sample_nature)
print("ตัวอย่าง sample_architectural (encoded):", sample_architectural)
print("ตัวอย่าง sample_animal (encoded):", sample_animal)

จำนวนข้อมูลทั้งหมด: 2888
จำนวนข้อมูลสำหรับ training: 2310
จำนวนข้อมูลสำหรับ validation: 578
ขนาดของรูปภาพ: torch.Size([3, 256, 256])
ตัวอย่าง sample_three_d (encoded): 1
ตัวอย่าง sample_symmetric (encoded): 0
ตัวอย่าง sample_food (encoded): 0
ตัวอย่าง sample_person (encoded): 1
ตัวอย่าง sample_nature (encoded): 0
ตัวอย่าง sample_architectural (encoded): 0
ตัวอย่าง sample_animal (encoded): 0


In [189]:
class ImageLabelModel(nn.Module):
    def __init__(self, num_three_d, num_symmetric, num_food, num_person ,num_nature ,num_architectural ,num_animal):
        super(ImageLabelModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
        )
        self.three_d_classifier = nn.Linear(512, num_three_d)
        self.symmetric_classifier = nn.Linear(512, num_symmetric)
        self.food_classifier = nn.Linear(512, num_food)
        self.person_classifier = nn.Linear(512, num_person)
        self.nature_classifier = nn.Linear(512, num_nature)
        self.architectural_classifier = nn.Linear(512, num_architectural)
        self.animal_classifier = nn.Linear(512, num_animal)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        three_d = self.three_d_classifier(x)
        symmetric = self.symmetric_classifier(x)
        food = self.food_classifier(x)
        person = self.person_classifier(x)
        nature = self.nature_classifier(x)
        architectural = self.architectural_classifier(x)
        animal = self.animal_classifier(x)
        return three_d, symmetric, food, person ,nature ,architectural ,animal

# Create the model instance
model = ImageLabelModel(
    num_three_d=len(dataset.three_d_encoder.classes_),
    num_symmetric=len(dataset.symmetric_encoder.classes_),
    num_food=len(dataset.food_encoder.classes_),
    num_person=len(dataset.person_encoder.classes_),
    num_nature=len(dataset.nature_encoder.classes_),
    num_architectural=len(dataset.architectural_encoder.classes_),
    num_animal=len(dataset.animal_encoder.classes_),
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(model)

ImageLabelModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (classifier): Sequential(
    (0): Linear(in_features=512, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): ReL

In [190]:
# Cell 4: Define loss function and optimizer
def weighted_loss(three_d_out, symmetric_out, food_out, person_out ,nature_out ,architectural_out ,animal_out, three_d, symmetric, food, person ,nature ,architectural ,animal):
    three_d_loss = criterion(three_d_out, three_d)
    symmetric_loss = criterion(symmetric_out, symmetric)
    food_loss = criterion(food_out, food)
    person_loss = criterion(person_out, person)
    nature_loss = criterion(nature_out, nature)
    architectural_loss = criterion(architectural_out, architectural)
    animal_loss = criterion(animal_out, animal)
    return 0.15 * three_d_loss + 0.14 * symmetric_loss + 0.14 * food_loss + 0.14 * person_loss + 0.15 * nature_loss + 0.14 * architectural_loss + 0.14 * animal_loss

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

/home/camp-ai-28/miniconda3/envs/env/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [191]:
# Cell 5: Training loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, three_d, symmetric, food, person ,nature ,architectural ,animal in train_loader:
        images, three_d, symmetric, food, person ,nature ,architectural ,animal = images.to(device), three_d.to(device), symmetric.to(device), food.to(device), person.to(device) ,nature.to(device) ,architectural.to(device) ,animal.to(device)

        optimizer.zero_grad()

        three_d_out, symmetric_out, food_out, person_out ,nature_out ,architectural_out ,animal_out = model(images)
        loss = weighted_loss(three_d_out, symmetric_out, food_out, person_out ,nature_out ,architectural_out ,animal_out, three_d, symmetric, food, person ,nature ,architectural ,animal)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    correct_three_d, correct_symmetric, correct_food, correct_person ,correct_nature ,correct_architectural ,correct_animal = 0, 0, 0, 0, 0, 0, 0
    total = 0

    with torch.no_grad():
        for images, three_d, symmetric, food, person ,nature ,architectural ,animal in val_loader:
            images, three_d, symmetric, food, person ,nature ,architectural ,animal = images.to(device), three_d.to(device), symmetric.to(device), food.to(device), person.to(device) ,nature.to(device) ,architectural.to(device) ,animal.to(device)

            three_d_out, symmetric_out, food_out, person_out ,nature_out ,architectural_out ,animal_out = model(images)
            loss = weighted_loss(three_d_out, symmetric_out, food_out, person_out ,nature_out ,architectural_out ,animal_out, three_d, symmetric, food, person ,nature ,architectural ,animal)

            val_loss += loss.item()

            _, predicted_three_d = torch.max(three_d_out, 1)
            _, predicted_symmetric = torch.max(symmetric_out, 1)
            _, predicted_food = torch.max(food_out, 1)
            _, predicted_person = torch.max(person_out, 1)
            _, predicted_nature = torch.max(nature_out, 1)
            _, predicted_architectural = torch.max(architectural_out, 1)
            _, predicted_animal = torch.max(animal_out, 1)

            total += three_d.size(0)
            correct_three_d += (predicted_three_d == three_d).sum().item()
            correct_symmetric += (predicted_symmetric == symmetric).sum().item()
            correct_food += (predicted_food == food).sum().item()
            correct_person += (predicted_person == person).sum().item()
            correct_nature += (predicted_nature == nature).sum().item()
            correct_architectural += (predicted_architectural == architectural).sum().item()
            correct_animal += (predicted_animal == animal).sum().item()

    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Training loss: {running_loss/len(train_loader):.4f}')
    print(f'Validation loss: {val_loss/len(val_loader):.4f}')
    print(f'3D accuracy: {100 * correct_three_d / total:.2f}%')
    print(f'Symmetric accuracy: {100 * correct_symmetric / total:.2f}%')
    print(f'Food accuracy: {100 * correct_food / total:.2f}%')
    print(f'Person accuracy: {100 * correct_person / total:.2f}%')
    print(f'Nature accuracy: {100 * correct_nature / total:.2f}%')
    print(f'Architectural accuracy: {100 * correct_architectural / total:.2f}%')
    print(f'Animal accuracy: {100 * correct_animal / total:.2f}%')
    print('-' * 60)

    scheduler.step(val_loss)

print('Finished Training')

Epoch 1/100
Training loss: 0.4138
Validation loss: 0.3634
3D accuracy: 79.24%
Symmetric accuracy: 95.67%
Food accuracy: 97.23%
Person accuracy: 55.54%
Nature accuracy: 60.73%
Architectural accuracy: 98.96%
Animal accuracy: 92.39%
------------------------------------------------------------
Epoch 2/100
Training loss: 0.3720
Validation loss: 0.3268
3D accuracy: 79.24%
Symmetric accuracy: 95.67%
Food accuracy: 97.23%
Person accuracy: 64.36%
Nature accuracy: 70.24%
Architectural accuracy: 98.96%
Animal accuracy: 92.39%
------------------------------------------------------------
Epoch 3/100
Training loss: 0.3222
Validation loss: 0.3113
3D accuracy: 79.24%
Symmetric accuracy: 95.67%
Food accuracy: 97.23%
Person accuracy: 67.13%
Nature accuracy: 75.78%
Architectural accuracy: 98.96%
Animal accuracy: 92.39%
------------------------------------------------------------
Epoch 4/100
Training loss: 0.2892
Validation loss: 0.2755
3D accuracy: 79.24%
Symmetric accuracy: 95.67%
Food accuracy: 97.23%


In [192]:
# Cell 6: Save the model
torch.save(model.state_dict(), 'Day1/weight/image_label_model.pth')
print("Model saved successfully")

Model saved successfully


In [193]:
# Cell 7: Load and evaluate on test set
model = ImageLabelModel(
    num_three_d=len(dataset.three_d_encoder.classes_),
    num_symmetric=len(dataset.symmetric_encoder.classes_),
    num_food=len(dataset.food_encoder.classes_),
    num_person=len(dataset.person_encoder.classes_),
    num_nature=len(dataset.nature_encoder.classes_),
    num_architectural=len(dataset.architectural_encoder.classes_),
    num_animal=len(dataset.animal_encoder.classes_),
)
model.load_state_dict(torch.load('Day1/weight/image_label_model.pth'))
model = model.to(device)

def analyze_errors(model, data_loader):
    model.eval()
    three_d_errors, symmetric_errors, food_errors, person_errors ,nature_errors ,architectural_errors ,animal_errors = 0, 0, 0, 0, 0, 0, 0
    total = 0
    with torch.no_grad():
        for images, three_d, symmetric, food, person ,nature ,architectural ,animal in data_loader:
            images, three_d, symmetric, food, person ,nature ,architectural ,animal = images.to(device), three_d.to(device), symmetric.to(device), food.to(device), person.to(device) ,nature.to(device) ,architectural.to(device) ,animal.to(device)
            three_d_out, symmetric_out, food_out, person_out ,nature_out ,architectural_out ,animal_out = model(images)

            _, predicted_three_d = torch.max(three_d_out, 1)
            _, predicted_symmetric = torch.max(symmetric_out, 1)
            _, predicted_food = torch.max(food_out, 1)
            _, predicted_person = torch.max(person_out, 1)
            _, predicted_nature = torch.max(nature_out, 1)
            _, predicted_architectural = torch.max(architectural_out, 1)
            _, predicted_animal = torch.max(animal_out, 1)

            three_d_errors += (predicted_three_d != three_d).sum().item()
            symmetric_errors += (predicted_symmetric != symmetric).sum().item()
            food_errors += (predicted_food != food).sum().item()
            person_errors += (predicted_person != person).sum().item()
            nature_errors += (predicted_nature != nature).sum().item()
            architectural_errors += (predicted_architectural != architectural).sum().item()
            animal_errors += (predicted_animal != animal).sum().item()
            total += images.size(0)

    print(f"3D Error Rate: {three_d_errors/total:.2%}")
    print(f"Symmetric Error Rate: {symmetric_errors/total:.2%}")
    print(f"Food Error Rate: {food_errors/total:.2%}")
    print(f"Person Error Rate: {person_errors/total:.2%}")
    print(f"nature Error Rate: {nature_errors/total:.2%}")
    print(f"architectural Error Rate: {architectural_errors/total:.2%}")
    print(f"animal Error Rate: {animal_errors/total:.2%}")

test_dataset = ImageLabelDataset(csv_file='Day1/dataset1/NextGenClassification/sample_submission.csv', img_dir='Day1/dataset1/NextGenClassification', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

predictions = []

with torch.no_grad():
    for i, (images, _, _, _, _, _, _, _) in enumerate(test_loader):
        images = images.to(device)
        three_d_out, symmetric_out, food_out, person_out ,nature_out ,architectural_out ,animal_out = model(images)

        _, predicted_three_d = torch.max(three_d_out, 1)
        _, predicted_symmetric = torch.max(symmetric_out, 1)
        _, predicted_food = torch.max(food_out, 1)
        _, predicted_person = torch.max(person_out, 1)
        _, predicted_nature = torch.max(nature_out, 1)
        _, predicted_architectural = torch.max(architectural_out, 1)
        _, predicted_animal = torch.max(animal_out, 1)

        for j, (three_d, symmetric, food, person ,nature ,architectural ,animal) in enumerate(zip(predicted_three_d, predicted_symmetric, predicted_food, predicted_person, predicted_nature, predicted_architectural, predicted_animal)):
            index = i * test_loader.batch_size + j
            predictions.append({
                'image_path': test_dataset.data.iloc[index]['image_path'],
                '3D': dataset.three_d_encoder.inverse_transform([three_d.item()])[0],
                'symmetric': dataset.symmetric_encoder.inverse_transform([symmetric.item()])[0],
                'food': dataset.food_encoder.inverse_transform([food.item()])[0],
                'person': dataset.person_encoder.inverse_transform([person.item()])[0],
                'nature': dataset.nature_encoder.inverse_transform([nature.item()])[0],
                'architectural': dataset.architectural_encoder.inverse_transform([architectural.item()])[0],
                'animal': dataset.animal_encoder.inverse_transform([animal.item()])[0],
            })

original_df = pd.read_csv('Day1/dataset1/NextGenClassification/sample_submission.csv')

results_df = pd.DataFrame(predictions)

original_df['3D'] = results_df['3D']
original_df['symmetric'] = results_df['symmetric']
original_df['food'] = results_df['food']
original_df['person'] = results_df['person']
original_df['nature'] = results_df['nature']
original_df['architectural'] = results_df['architectural']
original_df['animal'] = results_df['animal']

original_df.to_csv("Day1/submission/waroon_submission.csv",index=False)
print("Day1/submission/waroon_submission.csv")

/tmp/ipykernel_3603222/3769205738.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Day1/weight/image_label_model.pth'))


Day1/submission/waroon_submission.csv


In [195]:
analyze_errors(model,val_loader)

3D Error Rate: 17.82%
Symmetric Error Rate: 4.50%
Food Error Rate: 2.42%
Person Error Rate: 9.00%
nature Error Rate: 6.23%
architectural Error Rate: 1.04%
animal Error Rate: 7.79%


In [187]:
# # Cell test
# y_predicted_three_d = []
# y_true_three_d = []

# y_predicted_symmetric = []
# y_true_symmetric = []

# y_predicted_food = []
# y_true_food = []

# y_predicted_person = []
# y_true_person = []

# y_predicted_nature = []
# y_true_nature = []

# y_predicted_architectural = []
# y_true_architectural = []

# y_predicted_animal = []
# y_true_animal = []


# model = ImageLabelModel(
#     num_three_d=len(dataset.three_d_encoder.classes_),
#     num_symmetric=len(dataset.symmetric_encoder.classes_),
#     num_food=len(dataset.food_encoder.classes_),
#     num_person=len(dataset.person_encoder.classes_),
#     num_nature=len(dataset.nature_encoder.classes_),
#     num_architectural=len(dataset.architectural_encoder.classes_),
#     num_animal=len(dataset.animal_encoder.classes_),
# )
# model.load_state_dict(torch.load('Day1/weight/image_label_model.pth'))
# model = model.to(device)

# def fbeta_submission(model,data_loader):
#     with torch.no_grad():
#         for images, three_d, symmetric, food, person ,nature ,architectural ,animal in data_loader:
#             images, three_d, symmetric, food, person ,nature ,architectural ,animal = images.to(device), three_d.to(device), symmetric.to(device), food.to(device), person.to(device) ,nature.to(device) ,architectural.to(device) ,animal.to(device)
#             three_d_out, symmetric_out, food_out, person_out ,nature_out ,architectural_out ,animal_out = model(images)

#             _, predicted_three_d = torch.max(three_d_out, 1)
#             _, predicted_symmetric = torch.max(symmetric_out, 1)
#             _, predicted_food = torch.max(food_out, 1)
#             _, predicted_person = torch.max(person_out, 1)
#             _, predicted_nature = torch.max(nature_out, 1)
#             _, predicted_architectural = torch.max(architectural_out, 1)
#             _, predicted_animal = torch.max(animal_out, 1)

#             predicted_three_d = predicted_three_d.cpu().numpy()
#             three_d = three_d.cpu().numpy()
#             predicted_symmetric = predicted_symmetric.cpu().numpy()
#             symmetric = symmetric.cpu().numpy()
#             predicted_food = predicted_food.cpu().numpy()
#             food = food.cpu().numpy()
#             predicted_person = predicted_person.cpu().numpy()
#             person = person.cpu().numpy()
#             predicted_nature = predicted_nature.cpu().numpy()
#             nature = nature.cpu().numpy()
#             predicted_architectural = predicted_architectural.cpu().numpy()
#             architectural = architectural.cpu().numpy()
#             predicted_animal = predicted_animal.cpu().numpy()
#             animal = animal.cpu().numpy()

#             y_predicted_three_d.append(predicted_three_d)
#             y_true_three_d.append(three_d)
#             y_predicted_symmetric.append(predicted_symmetric)
#             y_true_symmetric.append(symmetric)
#             y_predicted_food.append(predicted_food)
#             y_true_food.append(food)
#             y_predicted_person.append(predicted_person)
#             y_true_person.append(person)
#             y_predicted_nature.append(predicted_nature)
#             y_true_nature.append(nature)
#             y_predicted_architectural.append(predicted_architectural)
#             y_true_architectural.append(architectural)
#             y_predicted_animal.append(predicted_animal)
#             y_true_animal.append(animal)

#         three_d_f_beta_score = fbeta_score(y_true_three_d, y_predicted_three_d, average='macro', beta=0.5)
#         symmetric_f_beta_score = fbeta_score(y_true_symmetric,y_predicted_symmetric,average='macro',beta=0.5)
#         food_f_beta_score = fbeta_score(y_true_food,y_predicted_food,average='macro',beta=0.5)
#         person_f_beta_score = fbeta_score(y_true_person,y_predicted_person,average='macro',beta=0.5)
#         nature_f_beta_score = fbeta_score(y_true_nature,y_predicted_nature,average='macro',beta=0.5)
#         architectural_f_beta_score = fbeta_score(y_true_architectural,y_predicted_architectural,average='macro',beta=0.5)
#         animal_f_beta_score = fbeta_score(y_true_animal,y_predicted_animal,average='macro',beta=0.5)

#         f_beta_score = (three_d_f_beta_score + symmetric_f_beta_score + food_f_beta_score + person_f_beta_score + nature_f_beta_score + architectural_f_beta_score + animal_f_beta_score ) / 7
#         print(f_beta_score)

# fbeta_submission(model,val_loader)

/tmp/ipykernel_3603222/3537070496.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Day1/weight/image_label_model.pth'))


ValueError: unknown is not supported